In [1]:
import torch

# Tensor

In [2]:
tensor0d = torch.tensor(1)
tensor1d = torch.tensor([1, 2, 3])
tensor2d = torch.tensor([[1, 2, 3], [4, 5, 6]])
tensor3d = torch.tensor([[[1, 2, 3], [4, 5, 6]], [[7, 8, 9], [10, 11, 12]]])

In [3]:
tensor0d, tensor1d, tensor2d, tensor3d

(tensor(1),
 tensor([1, 2, 3]),
 tensor([[1, 2, 3],
         [4, 5, 6]]),
 tensor([[[ 1,  2,  3],
          [ 4,  5,  6]],
 
         [[ 7,  8,  9],
          [10, 11, 12]]]))

In [4]:
tensor1d.dtype

torch.int64

In [5]:
tensor2d.shape

torch.Size([2, 3])

In [6]:
tensor3d.shape

torch.Size([2, 2, 3])

In [7]:
tensor2d.reshape(3, 2)

tensor([[1, 2],
        [3, 4],
        [5, 6]])

In [8]:
# more commonly used than reshape
tensor2d.view(3, 2)

tensor([[1, 2],
        [3, 4],
        [5, 6]])

In [9]:
tensor2d.T

tensor([[1, 4],
        [2, 5],
        [3, 6]])

In [10]:
tensor2d.matmul(tensor2d.T)

tensor([[14, 32],
        [32, 77]])

In [11]:
# alternatively, you can use the @ operator
tensor2d @ tensor2d.T

tensor([[14, 32],
        [32, 77]])

# Models as computation graphs

In [12]:
import torch.nn.functional as F

y = torch.tensor([1.0])
x1 = torch.tensor([1.1])
w1 = torch.tensor([2.2])
b = torch.tensor([0.0])

z = x1 * w1 + b
a = torch.sigmoid(z)

loss = F.binary_cross_entropy(a, y)
print(loss)

tensor(0.0852)


# Automatic differentiation made easy

In [13]:
from torch.autograd import grad

y = torch.tensor([1.0])
x1 = torch.tensor([1.1])
w1 = torch.tensor([2.2], requires_grad=True)
b = torch.tensor([0.0], requires_grad=True)

z = x1 * w1 + b
a = torch.sigmoid(z)
loss = F.binary_cross_entropy(a, y)

# retain_graph=True allows us to compute multiple gradients without recomputing the forward pass
grad_L_w1 = grad(loss, w1, retain_graph=True) 
grad_L_b = grad(loss, b, retain_graph=True) 

print(grad_L_w1)
print(grad_L_b)

(tensor([-0.0898]),)
(tensor([-0.0817]),)


In [14]:
loss.backward()
print(w1.grad)
print(b.grad)

tensor([-0.0898])
tensor([-0.0817])


# Implementing multilayer neural networks

In [15]:
class NeuralNetwork(torch.nn.Module):
    def __init__(self, num_inputs, num_outputs):
        super().__init__()

        self.layers = torch.nn.Sequential(

            # 1st hidden layer
            torch.nn.Linear(num_inputs, 30),
            torch.nn.ReLU(),

            # 2nd hidden layer
            torch.nn.Linear(30, 20),
            torch.nn.ReLU(),

            # output layer
            torch.nn.Linear(20, num_outputs),
        )

    def forward(self, x):
        logits = self.layers(x)
        return logits

In [16]:
model = NeuralNetwork(50, 3)
print(model)

NeuralNetwork(
  (layers): Sequential(
    (0): Linear(in_features=50, out_features=30, bias=True)
    (1): ReLU()
    (2): Linear(in_features=30, out_features=20, bias=True)
    (3): ReLU()
    (4): Linear(in_features=20, out_features=3, bias=True)
  )
)


In [17]:
num_params = sum(
    p.numel() for p in model.parameters() if p.requires_grad
)
print("Total number of trainable model parameters:", num_params)

Total number of trainable model parameters: 2213


In [18]:
print(model.layers[0].weight)


Parameter containing:
tensor([[-0.0671, -0.0745,  0.0564,  ..., -0.0318, -0.0432,  0.0698],
        [-0.0321,  0.0008,  0.1406,  ...,  0.1025,  0.0460,  0.0626],
        [-0.0145,  0.0494,  0.0405,  ..., -0.1197,  0.0487, -0.0269],
        ...,
        [-0.0846, -0.0980, -0.0177,  ...,  0.1191,  0.0073, -0.0508],
        [ 0.1225,  0.0263, -0.1013,  ...,  0.0914, -0.0165, -0.0453],
        [ 0.1184,  0.0998, -0.1390,  ...,  0.0961, -0.0775, -0.0713]],
       requires_grad=True)


In [19]:
print(model.layers[0].weight.shape)


torch.Size([30, 50])


In [20]:
torch.manual_seed(123)

model = NeuralNetwork(50, 3)
print(model.layers[0].weight)

Parameter containing:
tensor([[-0.0577,  0.0047, -0.0702,  ...,  0.0222,  0.1260,  0.0865],
        [ 0.0502,  0.0307,  0.0333,  ...,  0.0951,  0.1134, -0.0297],
        [ 0.1077, -0.1108,  0.0122,  ...,  0.0108, -0.1049, -0.1063],
        ...,
        [-0.0787,  0.1259,  0.0803,  ...,  0.1218,  0.1303, -0.1351],
        [ 0.1359,  0.0175, -0.0673,  ...,  0.0674,  0.0676,  0.1058],
        [ 0.0790,  0.1343, -0.0293,  ...,  0.0344, -0.0971, -0.0509]],
       requires_grad=True)


In [21]:
torch.manual_seed(123)

X = torch.rand((1, 50))
out = model(X)
print(out)

tensor([[-0.1262,  0.1080, -0.1792]], grad_fn=<AddmmBackward0>)


In [22]:
torch.manual_seed(123)

X = torch.rand((1, 50))
out = model(X)
print(out)

tensor([[-0.1262,  0.1080, -0.1792]], grad_fn=<AddmmBackward0>)


In [23]:
with torch.no_grad():
    out = model(X)
print(out)

tensor([[-0.1262,  0.1080, -0.1792]])


In [24]:
with torch.no_grad():
    out = torch.softmax(model(X), dim=1)
print(out)

tensor([[0.3113, 0.3934, 0.2952]])
